In [1]:
import cv2
import kornia as K
import kornia.feature as KF
import matplotlib.pyplot as plt
import numpy as np
import torch
from kornia_moons.viz import draw_LAF_matches
from kornia.feature import LoFTR, LightGlue
import plotly.graph_objects as go
import gradio as gr

import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

from scipy.interpolate import griddata

device = K.utils.get_cuda_or_mps_device_if_available()
print(device)


c:\dev\ai_hack\.venv\Lib\site-packages\kornia\feature\lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
cpu


In [2]:
def find_outliers(data):

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    up_thresh = Q3 + 1.5 * IQR
    under_thresh = Q1 - 1.5 * IQR
    indexes_up = np.where(data > up_thresh)[0]
    indexes_under = np.where(data < under_thresh)[0]

    return np.concatenate((indexes_under, indexes_up))

def add_intermediate_points(mkpts0, mkpts1):
  """
  Takes 3 near points (near by coordinates, not position in array) and places
  one more between them for both arrays.

  Args:
    mkpts0: Array of keypoints for image 1.
    mkpts1: Array of keypoints for image 2.

  Returns:
    Updated arrays mkpts0 and mkpts1 with added intermediate points.
  """

  distances = np.linalg.norm(mkpts0[:, None, :] - mkpts0[None, :, :], axis=2)
  near_indices = np.argwhere(distances < 10)  

  # Select 3 near points
  if len(near_indices) >= 3:
    idx1, idx2 = near_indices[0][:2]
    pt1 = mkpts0[idx1]
    pt2 = mkpts0[idx2]
    pt3 = mkpts0[near_indices[1][1]]

    # Calculate intermediate point
    new_pt = (pt1 + pt2 + pt3) / 3

    # Add the new point to mkpts0
    mkpts0 = np.vstack([mkpts0, new_pt])

    # Repeat for mkpts1
    pt1 = mkpts1[idx1]
    pt2 = mkpts1[idx2]
    pt3 = mkpts1[near_indices[1][1]]
    new_pt = (pt1 + pt2 + pt3) / 3
    mkpts1 = np.vstack([mkpts1, new_pt])

  return mkpts0, mkpts1

# Main function for interface
def loftr_and_depth(img1, img2, baseline, center_distance, distance_cam2_object, distance_cam1_object):
    device = K.utils.get_cuda_or_mps_device_if_available()

    img1 = torch.tensor(img1).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0
    img2 = torch.tensor(img2).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0
    
    img1 = K.geometry.resize(img1, (512, 512), antialias=True)
    img2 = K.geometry.resize(img2, (512, 512), antialias=True)

    # Feature matching using LoFTR
    matcher = LoFTR(pretrained="indoor_new")

    input_dict = {
        "image0": K.color.rgb_to_grayscale(img1),
        "image1": K.color.rgb_to_grayscale(img2),
    }
    
    with torch.inference_mode():
        correspondences = matcher(input_dict)

    mkpts0 = correspondences["keypoints0"].cpu().numpy()
    mkpts1 = correspondences["keypoints1"].cpu().numpy()

    Fm, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)
    inliers = inliers > 0

    disparity = np.abs(mkpts0[:, 0] - mkpts1[:, 0])
    depth = (baseline * center_distance) / disparity

    # Create point cloud
    points = []
    for i in range(len(mkpts0)):
        x = mkpts0[i][0]
        y = mkpts0[i][1]
        z = depth[i]
        points.append([x, y, z])

    points = np.array(points)

    outliers_indexes = find_outliers(points[:, 2])
    points = np.delete(points, outliers_indexes, axis=0)

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    # Get the colors from the original images
    colors = []
    for i in range(len(mkpts0)):
        x = int(mkpts0[i][0])
        y = int(mkpts0[i][1])
        color = img1[0, :, y, x].cpu().numpy()
        colors.append(color)

    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=points[:, 0],
                y=points[:, 1],
                z=points[:, 2],
                mode="markers",
                marker=dict(
                    size=3,
                    color=colors, 
                ),
            )
        ]
    )

    # Extract x, y, z coordinates
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2] / 1000

    # Define grid over x and y
    grid_x, grid_y = np.mgrid[x.min():x.max(), y.min():y.max()]  # Adjust range and resolution as needed

    # Interpolate z values on the grid
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='linear')

    fig = go.Figure(data=[go.Surface(x=grid_x, y=grid_y, z=grid_z)])
    
    return fig




In [3]:
inputs = [
    gr.Image(type="numpy", label="Изображение 1"),
    gr.Image(type="numpy", label="Изображение 2"),
    gr.Number(value=412, label="Базовое расстояние (мм)"),
    gr.Number(value=639, label="Расстояние до центра (мм)"),
    gr.Number(value=641, label="Расстояние от камеры 2 до объекта (мм)"),
    gr.Number(value=660, label="Расстояние от камеры 1 до объекта (мм)"),
]

outputs = [
    gr.Plot(label="3D облако точек"),
]

description = """
Загрузите пару изображений и при необходимости настройте параметры, затем нажмите 'Submit' для измерения геометрии.
- **Базовое расстояние**: Расстояние между двумя камерами (в мм).
- **Расстояние до центра**: Расстояние от центра между камерами до объекта (в мм).
- **Расстояние от камеры 2 до объекта**: Расстояние от камеры 2 (левая) до объекта (в мм).
- **Расстояние от камеры 1 до объекта**: Расстояние от камеры 1 (правая) до объекта (в мм).
"""

title = "Измерение геометрии на основе стереоизображений"

demo = gr.Interface(
    fn=loftr_and_depth,
    inputs=inputs,
    outputs=outputs,
    live=False,
    title=title,
    description=description
)


In [4]:
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
